In [1]:
import pandas as pd
import numpy as np
import anndata
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
import pathlib

warnings.filterwarnings('ignore')

In [2]:
from matplotlib import rc
labelsize = 6
linewidth = 0.6
rc('lines', linewidth=linewidth)
rc('axes', labelsize=labelsize, linewidth=linewidth)
rc('xtick', labelsize=labelsize)
rc('ytick', labelsize=labelsize)
rc('xtick.major', width=linewidth)
rc('ytick.major', width=linewidth)
rc('xtick.minor', width=linewidth - 0.2)
rc('ytick.minor', width=linewidth - 0.2)

In [3]:
chrom = 'chr2'
genes = ['Lhx6']
slop = 250000
n_pc = 10
resolution = 1

In [4]:
# Parameters
chrom = "chr13"
genes = ["ENSMUSG00000021366.8", "ENSMUSG00000021540.16", "ENSMUSG00000005583.16", "ENSMUSG00000021709.14", "ENSMUSG00000021700.10", "ENSMUSG00000058022.14", "ENSMUSG00000056222.15", "ENSMUSG00000021676.10", "ENSMUSG00000025876.15", "ENSMUSG00000021708.17", "ENSMUSG00000049985.15", "ENSMUSG00000021338.17", "ENSMUSG00000021466.12", "ENSMUSG00000057531.14", "ENSMUSG00000021754.17", "ENSMUSG00000052551.16", "ENSMUSG00000021756.12", "ENSMUSG00000014850.15", "ENSMUSG00000052374.16", "ENSMUSG00000021730.8", "ENSMUSG00000041417.15", "ENSMUSG00000085936.7", "ENSMUSG00000021536.7", "ENSMUSG00000033102.15", "ENSMUSG00000060176.4", "ENSMUSG00000039004.6", "ENSMUSG00000021725.9", "ENSMUSG00000021373.16", "ENSMUSG00000021313.16", "ENSMUSG00000044164.2", "ENSMUSG00000113772.1", "ENSMUSG00000021541.14", "ENSMUSG00000041112.17", "ENSMUSG00000038175.10", "ENSMUSG00000021375.11", "ENSMUSG00000021684.17", "ENSMUSG00000034987.4", "ENSMUSG00000021340.13", "ENSMUSG00000054728.16", "ENSMUSG00000052981.6", "ENSMUSG00000003992.15", "ENSMUSG00000034751.16", "ENSMUSG00000047842.7", "ENSMUSG00000021552.7", "ENSMUSG00000051111.16", "ENSMUSG00000114767.1", "ENSMUSG00000042348.10", "ENSMUSG00000009734.18", "ENSMUSG00000001542.7", "ENSMUSG00000021710.11", "ENSMUSG00000021699.17", "ENSMUSG00000021556.12", "ENSMUSG00000113573.1", "ENSMUSG00000047094.9", "ENSMUSG00000021662.11", "ENSMUSG00000068184.7", "ENSMUSG00000046169.10", "ENSMUSG00000021686.6", "ENSMUSG00000049538.14", "ENSMUSG00000036006.19", "ENSMUSG00000021420.13", "ENSMUSG00000021391.10", "ENSMUSG00000042426.6", "ENSMUSG00000038267.15", "ENSMUSG00000113074.1", "ENSMUSG00000021303.14", "ENSMUSG00000000078.7", "ENSMUSG00000114276.1", "ENSMUSG00000113856.1", "ENSMUSG00000021638.12", "ENSMUSG00000051335.6", "ENSMUSG00000021460.17", "ENSMUSG00000005397.8", "ENSMUSG00000114121.1", "ENSMUSG00000021451.16", "ENSMUSG00000087143.9", "ENSMUSG00000113701.1", "ENSMUSG00000069170.14", "ENSMUSG00000038991.17", "ENSMUSG00000047419.5", "ENSMUSG00000021719.9", "ENSMUSG00000055254.15", "ENSMUSG00000021615.13", "ENSMUSG00000021474.9", "ENSMUSG00000078952.9", "ENSMUSG00000006191.17", "ENSMUSG00000038068.15", "ENSMUSG00000015533.9", "ENSMUSG00000037946.11", "ENSMUSG00000039087.17", "ENSMUSG00000021322.8", "ENSMUSG00000021587.5", "ENSMUSG00000113017.1", "ENSMUSG00000021482.10", "ENSMUSG00000034488.14", "ENSMUSG00000035936.6", "ENSMUSG00000113919.1", "ENSMUSG00000021311.9", "ENSMUSG00000047789.5", "ENSMUSG00000046876.16", "ENSMUSG00000021614.16", "ENSMUSG00000016756.17", "ENSMUSG00000021589.13", "ENSMUSG00000078945.10", "ENSMUSG00000038732.15", "ENSMUSG00000017756.9", "ENSMUSG00000113553.1", "ENSMUSG00000021720.11", "ENSMUSG00000021553.14", "ENSMUSG00000021478.6", "ENSMUSG00000114038.1", "ENSMUSG00000045034.12", "ENSMUSG00000041685.16", "ENSMUSG00000038132.6", "ENSMUSG00000021423.6", "ENSMUSG00000100733.2", "ENSMUSG00000118436.1", "ENSMUSG00000100596.1", "ENSMUSG00000113404.1", "ENSMUSG00000052727.6", "ENSMUSG00000113794.1", "ENSMUSG00000113825.1", "ENSMUSG00000021640.6", "ENSMUSG00000114743.1", "ENSMUSG00000007617.18", "ENSMUSG00000021461.17", "ENSMUSG00000021646.9", "ENSMUSG00000114372.1", "ENSMUSG00000097622.2", "ENSMUSG00000114590.1", "ENSMUSG00000021318.15", "ENSMUSG00000035910.15", "ENSMUSG00000047324.5", "ENSMUSG00000097098.8", "ENSMUSG00000021671.9", "ENSMUSG00000114841.1", "ENSMUSG00000114213.1", "ENSMUSG00000021715.12", "ENSMUSG00000038025.8", "ENSMUSG00000087460.2", "ENSMUSG00000021364.16", "ENSMUSG00000021534.8", "ENSMUSG00000047117.12", "ENSMUSG00000114731.1", "ENSMUSG00000021196.14", "ENSMUSG00000014164.15", "ENSMUSG00000109853.1", "ENSMUSG00000118445.1", "ENSMUSG00000097182.2", "ENSMUSG00000113982.1", "ENSMUSG00000021575.16", "ENSMUSG00000021608.15", "ENSMUSG00000089940.1", "ENSMUSG00000041773.8", "ENSMUSG00000114715.1", "ENSMUSG00000085620.2", "ENSMUSG00000021696.9", "ENSMUSG00000113244.1", "ENSMUSG00000109228.2", "ENSMUSG00000053101.3", "ENSMUSG00000021624.9", "ENSMUSG00000097399.7", "ENSMUSG00000021448.7", "ENSMUSG00000051758.6", "ENSMUSG00000114646.1", "ENSMUSG00000114536.1", "ENSMUSG00000021371.5", "ENSMUSG00000025867.8", "ENSMUSG00000050334.12", "ENSMUSG00000044951.15", "ENSMUSG00000038286.11", "ENSMUSG00000041324.14", "ENSMUSG00000044566.15", "ENSMUSG00000113121.1", "ENSMUSG00000021365.15", "ENSMUSG00000097009.2", "ENSMUSG00000038372.14", "ENSMUSG00000091387.2", "ENSMUSG00000046573.7", "ENSMUSG00000115791.1", "ENSMUSG00000042284.10", "ENSMUSG00000101136.1", "ENSMUSG00000114437.1", "ENSMUSG00000114438.1", "ENSMUSG00000021301.9", "ENSMUSG00000042015.17", "ENSMUSG00000021336.17", "ENSMUSG00000113816.1", "ENSMUSG00000110235.1", "ENSMUSG00000021613.9", "ENSMUSG00000021411.16", "ENSMUSG00000021470.19", "ENSMUSG00000113358.1", "ENSMUSG00000021567.15", "ENSMUSG00000021759.14", "ENSMUSG00000050919.9", "ENSMUSG00000097101.3", "ENSMUSG00000114531.1", "ENSMUSG00000069255.13", "ENSMUSG00000114319.1", "ENSMUSG00000048264.16", "ENSMUSG00000089710.1", "ENSMUSG00000079048.8", "ENSMUSG00000084788.2", "ENSMUSG00000043631.8", "ENSMUSG00000114407.1", "ENSMUSG00000036721.14", "ENSMUSG00000034334.8", "ENSMUSG00000035139.14", "ENSMUSG00000048978.14", "ENSMUSG00000113543.1", "ENSMUSG00000114613.1", "ENSMUSG00000107008.3", "ENSMUSG00000113073.1", "ENSMUSG00000021476.9", "ENSMUSG00000021669.15", "ENSMUSG00000021668.15", "ENSMUSG00000021360.16", "ENSMUSG00000064138.14", "ENSMUSG00000114838.1", "ENSMUSG00000100727.6", "ENSMUSG00000113211.1", "ENSMUSG00000116207.1", "ENSMUSG00000021702.7", "ENSMUSG00000025453.18", "ENSMUSG00000114242.1", "ENSMUSG00000021716.14", "ENSMUSG00000069171.14", "ENSMUSG00000036123.8", "ENSMUSG00000055137.7", "ENSMUSG00000038080.17", "ENSMUSG00000021758.14", "ENSMUSG00000114493.2", "ENSMUSG00000021732.14", "ENSMUSG00000113742.1", "ENSMUSG00000113960.1", "ENSMUSG00000112941.1", "ENSMUSG00000115697.2", "ENSMUSG00000114790.1", "ENSMUSG00000021565.16", "ENSMUSG00000086649.1", "ENSMUSG00000021557.15", "ENSMUSG00000021337.8", "ENSMUSG00000041632.7", "ENSMUSG00000042364.11", "ENSMUSG00000000253.13", "ENSMUSG00000021621.15", "ENSMUSG00000113591.1", "ENSMUSG00000113810.1", "ENSMUSG00000114596.1", "ENSMUSG00000021377.15", "ENSMUSG00000114854.1", "ENSMUSG00000114457.1", "ENSMUSG00000006711.15", "ENSMUSG00000086363.7", "ENSMUSG00000021357.14", "ENSMUSG00000113186.1", "ENSMUSG00000021619.6", "ENSMUSG00000114961.1", "ENSMUSG00000021728.8", "ENSMUSG00000104214.1", "ENSMUSG00000069227.4", "ENSMUSG00000021680.8", "ENSMUSG00000090016.1", "ENSMUSG00000113727.1", "ENSMUSG00000113484.1", "ENSMUSG00000095125.2", "ENSMUSG00000115020.1", "ENSMUSG00000036110.14", "ENSMUSG00000085759.1", "ENSMUSG00000037989.15", "ENSMUSG00000113680.1", "ENSMUSG00000025878.16", "ENSMUSG00000095105.2", "ENSMUSG00000078942.10", "ENSMUSG00000021427.10", "ENSMUSG00000114478.1", "ENSMUSG00000099639.6", "ENSMUSG00000086531.2", "ENSMUSG00000008859.8", "ENSMUSG00000083653.2", "ENSMUSG00000114649.1", "ENSMUSG00000113759.1", "ENSMUSG00000113717.1", "ENSMUSG00000097242.1", "ENSMUSG00000096537.6", "ENSMUSG00000114348.1", "ENSMUSG00000113463.1", "ENSMUSG00000110055.1", "ENSMUSG00000114307.1", "ENSMUSG00000005312.15", "ENSMUSG00000109561.1", "ENSMUSG00000033715.15", "ENSMUSG00000054266.2", "ENSMUSG00000042043.6", "ENSMUSG00000061829.4", "ENSMUSG00000113758.1", "ENSMUSG00000003062.13", "ENSMUSG00000046932.2", "ENSMUSG00000102635.1", "ENSMUSG00000113737.1", "ENSMUSG00000114273.1", "ENSMUSG00000113689.1", "ENSMUSG00000114014.1", "ENSMUSG00000038042.10", "ENSMUSG00000021697.12", "ENSMUSG00000069296.4", "ENSMUSG00000114695.1", "ENSMUSG00000113313.1", "ENSMUSG00000021612.15", "ENSMUSG00000090907.1", "ENSMUSG00000113770.1", "ENSMUSG00000114534.1", "ENSMUSG00000114401.1", "ENSMUSG00000021666.16", "ENSMUSG00000113057.1", "ENSMUSG00000114206.1", "ENSMUSG00000021214.14", "ENSMUSG00000021620.4", "ENSMUSG00000043183.18", "ENSMUSG00000114483.1", "ENSMUSG00000021216.7", "ENSMUSG00000071203.6", "ENSMUSG00000042655.5", "ENSMUSG00000021573.15", "ENSMUSG00000021609.6", "ENSMUSG00000114929.1", "ENSMUSG00000069295.2", "ENSMUSG00000094898.2", "ENSMUSG00000016477.18", "ENSMUSG00000113324.1", "ENSMUSG00000049115.15", "ENSMUSG00000097709.8", "ENSMUSG00000009470.17", "ENSMUSG00000113175.1", "ENSMUSG00000113744.1", "ENSMUSG00000021335.13", "ENSMUSG00000098087.1", "ENSMUSG00000113269.1", "ENSMUSG00000114017.1", "ENSMUSG00000087603.1", "ENSMUSG00000056749.7", "ENSMUSG00000113326.1", "ENSMUSG00000114134.1", "ENSMUSG00000041138.16", "ENSMUSG00000021675.4", "ENSMUSG00000114198.1", "ENSMUSG00000044934.13", "ENSMUSG00000063998.4", "ENSMUSG00000081903.1", "ENSMUSG00000089854.2", "ENSMUSG00000087202.1", "ENSMUSG00000113998.1", "ENSMUSG00000114637.1", "ENSMUSG00000021147.17", "ENSMUSG00000021431.15", "ENSMUSG00000113216.1", "ENSMUSG00000098107.1", "ENSMUSG00000114422.1", "ENSMUSG00000114882.1", "ENSMUSG00000114710.1", "ENSMUSG00000110016.1", "ENSMUSG00000114843.1", "ENSMUSG00000114474.1", "ENSMUSG00000100410.1", "ENSMUSG00000113557.1", "ENSMUSG00000099917.2", "ENSMUSG00000113470.1", "ENSMUSG00000069248.11", "ENSMUSG00000069289.2", "ENSMUSG00000057069.7", "ENSMUSG00000021417.15", "ENSMUSG00000087335.2", "ENSMUSG00000114846.1", "ENSMUSG00000113241.1", "ENSMUSG00000035367.9", "ENSMUSG00000074867.4", "ENSMUSG00000021687.14", "ENSMUSG00000082956.4", "ENSMUSG00000051098.6", "ENSMUSG00000078926.8", "ENSMUSG00000114605.1", "ENSMUSG00000051606.5", "ENSMUSG00000054944.7", "ENSMUSG00000021403.3", "ENSMUSG00000071551.13", "ENSMUSG00000114104.1", "ENSMUSG00000114487.1", "ENSMUSG00000114290.1", "ENSMUSG00000114525.1", "ENSMUSG00000113295.1", "ENSMUSG00000114473.1", "ENSMUSG00000021363.14", "ENSMUSG00000114959.1", "ENSMUSG00000097715.2", "ENSMUSG00000042842.11", "ENSMUSG00000021493.15", "ENSMUSG00000113626.1", "ENSMUSG00000021611.9", "ENSMUSG00000114036.1", "ENSMUSG00000021468.8", "ENSMUSG00000069805.10"]
slop = 250000
n_pc = 10
resolution = 1


In [5]:
output_dir = pathlib.Path(chrom)
output_dir.mkdir(exist_ok=True)

## Cell Meta

In [6]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)
use_clusters = [
    i.replace(' ', '_') for i in cell_tidy_data[cell_tidy_data['CellClass'].isin(['Exc', 'Inh'])]
    ['SubType'].unique() if 'Outlier' not in i
]
len(use_clusters)

145

## ATAC peaks

In [7]:
atac_peak = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/study/DMRCluster/SubType.ATAC_peak_merged.msg')
atac_peak = atac_peak.loc[atac_peak.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

## Gene Info

In [8]:
gene_meta = pd.read_csv(
    '/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
    index_col='gene_id', sep='\t'
)
gene_meta = gene_meta[gene_meta['chrom'] == chrom].copy()

In [9]:
exon_bed = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm22/genome_anno/exon.all.bed',
                       header=None, sep='\t')
exon_bed.columns = ['chrom', 'start', 'end', 'gene_id', 'gene_name']

## DMR Info

In [10]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5', 'r') as hdf:
    dmr_rate = hdf['Rate']
dmr_rate = dmr_rate.loc[dmr_rate.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

dmr_corr = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/DMRGeneCorr/TotalGeneDMRCorrLoop.0.3.msg'
)
dmr_corr = dmr_corr.set_index(['DMR', 'Gene'])
dmr_corr = dmr_corr.loc[dmr_corr.index.get_level_values('DMR').isin(dmr_rate.index)].copy()

dmr_bed = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalDMR.nofilter.bed',
                      sep='\t', header=None, index_col=3)
dmr_bed.columns = ['chrom', 'start', 'end']
dmr_bed = dmr_bed[dmr_bed['chrom'] == chrom].copy()

dmr_hits = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad')
dmr_hits = dmr_hits[dmr_rate.index, :].copy()
dmr_hits = dmr_hits[:, use_clusters].copy()

dmr_annot = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRAnnotation.h5ad')
dmr_annot = dmr_annot[dmr_rate.index, :].copy()

In [11]:
dmr_hits.shape[0]

224117

In [12]:
def get_gene(gene_id):
    _gene = gene_meta.loc[gene_id]
    return _gene.name, _gene['chrom'], _gene['start'], _gene['end'], _gene['strand']

## Gene's DMR clustering

In [13]:
def calculate_gene(gene_id):
    # get gene information
    gene_id, _, gene_start, gene_end, _ = get_gene(gene_id)

    # select related DMRs
    related_dmr = dmr_bed[(dmr_bed['start'] > gene_start - slop) &
                          (dmr_bed['end'] < gene_end + slop)].copy()
    
    related_dmr_rate = dmr_rate.loc[related_dmr.index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    
    # construct Adata
    adata = anndata.AnnData(X=related_dmr_rate.values.copy(),
                            obs=pd.DataFrame([], related_dmr_rate.index),
                            var=pd.DataFrame([], related_dmr_rate.columns))
    sc.pp.scale(adata)
    sc.pp.pca(adata)
    
    pcs = adata.obsm['X_pca'][:, :n_pc]
    sc.pp.neighbors(adata, n_neighbors=int(round(np.log2(adata.shape[0]))), n_pcs=n_pc)
    sc.tl.leiden(adata, resolution=resolution)
    return adata.obs


def get_annotation(gene_id):
    gene_id, _, gene_start, gene_end, strand = get_gene(gene_id)
    tss = gene_start if strand == '+' else gene_end
    gene_cluster = calculate_gene(gene_id)
    this_corr = dmr_corr[dmr_corr.index.get_level_values('Gene') == gene_id]['Corr']
    this_corr.index = this_corr.index.droplevel('Gene')
    gene_cluster['Corr'] = gene_cluster.index.map(this_corr).fillna(0)
    
    this_dmr_bed = dmr_bed.loc[gene_cluster.index]
    dmr_center = (this_dmr_bed['end'] + this_dmr_bed['start']) / 2
    gene_length = gene_end - gene_start
    if strand == '+':
        gene_cluster['reldist_tss'] = (dmr_center - gene_start) / gene_length
    else:
        gene_cluster['reldist_tss'] = (gene_end - dmr_center) / gene_length
    gene_cluster['in_gene_body'] = (gene_cluster['reldist_tss'] > 0) & (gene_cluster['reldist_tss'] < 1)
    
    this_annot = dmr_annot[gene_cluster.index]
    annot_df = pd.DataFrame(this_annot.X.todense(), 
                 index=this_annot.obs_names, columns=this_annot.var_names)
    
    # annotate TE cols
    dna_te = annot_df.columns[20:33]
    gene_cluster['is_dna_te'] = annot_df[dna_te].sum(axis=1) != 0
    
    line_te = annot_df.columns[33:39]
    gene_cluster['is_line_te'] = annot_df[line_te].sum(axis=1) != 0
    
    ltr_te = annot_df.columns[39:45]
    gene_cluster['is_ltr_te'] = annot_df[ltr_te].sum(axis=1) != 0
    
    sine_te = annot_df.columns[45:52]
    gene_cluster['is_sine_te'] = annot_df[sine_te].sum(axis=1) != 0
    
    # this dmr within GOI's gene feature
    gene_cluster['in_intron'] = annot_df['intron'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_exon'] = annot_df['exon'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr3'] = annot_df['UTR3'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr5'] = annot_df['UTR5'].astype(bool) & gene_cluster['in_gene_body']
    
    # previous mC study
    gene_cluster['feDMR'] = annot_df['feDMR'].astype(bool)
    gene_cluster['adultDMR'] = annot_df['adultDMR'].astype(bool)
    
    
    other_profiles = []
    # DMR hypo call in each cluster
    this_hypo_hits = dmr_hits[gene_cluster.index]
    hits_df = pd.DataFrame(this_hypo_hits.X.todense(), 
                 index=this_hypo_hits.obs_names, columns=this_hypo_hits.var_names)
    hits_df.columns = hits_df.columns.map(lambda i: f'HypoDMR.{i}')
    other_profiles.append(hits_df)
    
    # DMR rate
    related_dmr_rate = dmr_rate.loc[gene_cluster['leiden'].sort_values().index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    related_dmr_rate.columns = related_dmr_rate.columns.map(lambda i: f'DMRRate.{i}')
    other_profiles.append(related_dmr_rate)
    
    # atac peak
    atac_peak_df = atac_peak.loc[related_dmr_rate.index, use_clusters].copy()
    other_profiles.append(atac_peak_df)
    atac_peak_df.columns = atac_peak_df.columns.map(lambda i: f'ATACPeak.{i}')
    
    dmr_annotation = pd.concat([gene_cluster] + other_profiles, axis=1, sort=True)
    return dmr_annotation

In [ ]:
for gene in genes:
    print(gene)
    check_path = output_dir / f'{gene}.DMR_cluster.msg'
    if check_path.exists():
        continue
    
    dmr_annotation = get_annotation(gene)
    dmr_annotation.to_msgpack(output_dir / f'{gene}.DMR_detail.msg', compress='zlib')
    
    cluster_annotation = dmr_annotation.groupby('leiden').mean()
    cluster_annotation.to_msgpack(output_dir / f'{gene}.DMR_cluster.msg', compress='zlib')

ENSMUSG00000021366.8
ENSMUSG00000021540.16
ENSMUSG00000005583.16
ENSMUSG00000021709.14
ENSMUSG00000021700.10
ENSMUSG00000058022.14
ENSMUSG00000056222.15
ENSMUSG00000021676.10
ENSMUSG00000025876.15
ENSMUSG00000021708.17
ENSMUSG00000049985.15
ENSMUSG00000021338.17
ENSMUSG00000021466.12
ENSMUSG00000057531.14
ENSMUSG00000021754.17
ENSMUSG00000052551.16
ENSMUSG00000021756.12
ENSMUSG00000014850.15
ENSMUSG00000052374.16
ENSMUSG00000021730.8
ENSMUSG00000041417.15
ENSMUSG00000085936.7
ENSMUSG00000021536.7
ENSMUSG00000033102.15
ENSMUSG00000060176.4
ENSMUSG00000039004.6
ENSMUSG00000021725.9
ENSMUSG00000021373.16
ENSMUSG00000021313.16
ENSMUSG00000044164.2
ENSMUSG00000113772.1
ENSMUSG00000021541.14
ENSMUSG00000041112.17
ENSMUSG00000038175.10
ENSMUSG00000021375.11
ENSMUSG00000021684.17
ENSMUSG00000034987.4
ENSMUSG00000021340.13
ENSMUSG00000054728.16
ENSMUSG00000052981.6
ENSMUSG00000003992.15
ENSMUSG00000034751.16
ENSMUSG00000047842.7
ENSMUSG00000021552.7
ENSMUSG00000051111.16
ENSMUSG00000114767.1
EN